In [1]:
import os
import time
import torch
from shutil import copyfile
from torchvision import transforms

In [3]:
train_dir = './train'
dev_dir = './dev'
test_dir = 'test'
dir_path = './lfw-aligned'

In [8]:
class DataSegregate:
    def __init__(self, dir_path, train_dir, dev_dir, test_dir):
        self.labels = None
        self.dir_path = dir_path
        self.train_dir = train_dir
        self.dev_dir = dev_dir
        self.test_dir = test_dir
        self.images_dict = {}
        self.train_dict = {}
        self.dev_dict = {}
        self.test_dict = {}
        self.train_count = 0
        self.dev_count = 0
        self.test_count = 0
        self.total_count = 0
        self.make_dict()
        self.split()
        self.make_dirs()
        
    def make_dict(self):
        self.labels = os.listdir(self.dir_path)
        for label in self.labels:
            path = os.path.join(self.dir_path, label)
            if 'DS_Store' in path:
                continue
            images = os.listdir(path)
            count = len(images)
            if count >= 20 and count <= 30:
                self.images_dict[label] = images
                self.total_count += count
    
    def split(self):
        for label, image_list in self.images_dict.items():
            num_images = len(image_list)
            hist = torch.histc(torch.range(0,num_images-1), bins = 2, min=0, max=num_images-1)
            num_train, num_dev_test = hist.type(torch.LongTensor).tolist()
            # make sure num_train = 80%, num_dev = num_test = 10%
            num_train, num_dev, num_test = self.even_split(num_train, num_dev_test)
            self.train_count += num_train
            self.dev_count += num_dev
            self.test_count += num_test
            self.train_dict[label] = image_list[:num_train]
            self.dev_dict[label] = image_list[num_train:num_train+num_dev]
            self.test_dict[label] = image_list[num_train+num_dev:]
    
    def even_split(self, num_train, num_dev_test):
        while num_dev_test/(num_train + num_dev_test) > 0.2 and num_dev_test > 2:
            num_dev_test -= 1
            num_train += 1
        hist = torch.histc(torch.range(0,num_dev_test-1), bins = 2, min=0, max=num_dev_test-1)
        num_dev, num_test = hist.type(torch.LongTensor).tolist()
        return num_train, num_dev, num_test
    
    def copy_files(self, data_dict, dir_output):
        if not os.path.exists(dir_output):
            os.makedirs(dir_output)
        
        for label, image_list in data_dict.items():
            label_path_target = os.path.join(dir_output, label)
            label_path_source = os.path.join(self.dir_path, label)
            if not os.path.exists(label_path_target):
                os.makedirs(label_path_target)
            for image in image_list:
                source_path = os.path.join(label_path_source, image)
                target_path = os.path.join(label_path_target, image)
                copyfile(source_path, target_path)
    
    def make_dirs(self):
        self.copy_files(self.train_dict, train_dir)
        self.copy_files(self.dev_dict, dev_dir)
        self.copy_files(self.test_dict, test_dir)

In [9]:
tic = time.time()
data_seg = DataSegregate(dir_path, train_dir, dev_dir, test_dir)
toc = time.time()
print('cost time: '+str(toc-tic)+' s')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].


cost time: 0.3979511260986328 s


In [10]:
data_seg.train_count, data_seg.dev_count, data_seg.test_count

(581, 62, 70)

In [11]:
data_seg.train_count/data_seg.total_count, data_seg.dev_count/data_seg.total_count, data_seg.test_count/data_seg.total_count

(0.814866760168303, 0.08695652173913043, 0.09817671809256662)